In [10]:
import io
import requests
import pandas as pd
from datetime import datetime, timezone
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import ipywidgets as widgets
from IPython.display import display, clear_output
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pytz


global session
session = requests.session()
retry = Retry(total=3, backoff_factor=1)
session.mount('http://', HTTPAdapter(max_retries=retry))
session.mount('https://', HTTPAdapter(max_retries=retry))
# Create input widgets
network_id_input = widgets.Text(
    value='1',
    description='Network ID:',
    disabled=False
)

market_contract_input = widgets.Text(
    value='0xa0ab94debb3cc9a7ea77f3205ba4ab23276fed08',
    description='Market Contract:',
    disabled=False
)

yt_contract_input = widgets.Text(
    value='0xbe05538f48d76504953c5d1068898c6642937427',
    description='YT Contract:',
    disabled=False
)

pt_contract_input = widgets.Text(
    value='0xee9085fc268f6727d5d4293dbabccf901ffdcc29',
    description='PT Contract:',
    disabled=False
)

# Create a button to trigger the data fetch
fetch_button = widgets.Button(description="Fetch Data")

# Output widget to display results
output = widgets.Output()

# Global variable to store the DataFrame
df = None

# Button click event handler
def on_button_click(b):
    global df
    global maturity;
    with output:
        clear_output()
        print("Fetching data...")
        
        # Retrieve input values
        network_id = network_id_input.value

        yt_contract = yt_contract_input.value
        pt_contract = pt_contract_input.value


        # Construct the request URL
        url = f'https://api-v2.pendle.finance/core/v1/{network_id}/assets/all'


        # Send request and parse response
        response = session.get(url)
        data = response.json()

        # Find valid YT assets
        valid_assets = [
            item for item in data
            if item.get('baseType') == 'YT' and
               item.get('address') == yt_contract
        ]

        if not valid_assets:
            print("No valid assets found with the given parameters")
            return

        symbol = valid_assets[0]['symbol']
        maturity = valid_assets[0]['expiry']

        # Fetch OHLCV data
        url_ohlcv_yteth = f'https://api-v2.pendle.finance/core/v4/{network_id}/prices/{pt_contract}/ohlcv'
        print(url_ohlcv_yteth)
        print(network_id)
        params = {
            "time_frame": "hour"
        }
        response = session.get(url_ohlcv_yteth, params=params)
        results = response.json().get('results', [])
        df = pd.read_csv(io.StringIO(results))
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.columns = df.columns.str.capitalize()
        
        print(f"Data fetched successfully for PT of{symbol}!")
        print(df.head())
        print("\nYou can now access the data using the global variable 'df'")

fetch_button.on_click(on_button_click)


# Display widgets
display(network_id_input, market_contract_input, yt_contract_input, pt_contract_input, fetch_button, output)

Text(value='1', description='Network ID:')

Text(value='0xa0ab94debb3cc9a7ea77f3205ba4ab23276fed08', description='Market Contract:')

Text(value='0xbe05538f48d76504953c5d1068898c6642937427', description='YT Contract:')

Text(value='0xee9085fc268f6727d5d4293dbabccf901ffdcc29', description='PT Contract:')

Button(description='Fetch Data', style=ButtonStyle())

Output()

In [11]:
network_id = network_id_input.value
market_contract = market_contract_input.value
params = {
    "time_frame": "hour",
}
url_apy = f'https://api-v2.pendle.finance/core/v1/{network_id}/markets/{market_contract}/apy-history-1ma'
response = session.get(url_apy, params=params)
if response.status_code == 200:
    data = response.json()
    if 'results' in data:
        csv_data = data['results']
        APY = pd.read_csv(io.StringIO(csv_data))
        APY['timestamp'] = pd.to_datetime(APY['timestamp'], unit='s', utc=True)
else:
    print(f"Error: {response.text}")

def convert_to_ohlc_4h(apy_df, pt_df):
    # Ensure both DataFrames are sorted by timestamp
    apy_df = apy_df.sort_values('timestamp')
    pt_df = pt_df.sort_values('Time')
    
    # Convert timestamp to datetime if it's not already and make it tz-aware
    apy_df['timestamp'] = pd.to_datetime(apy_df['timestamp'], utc=True)
    pt_df['Time'] = pd.to_datetime(pt_df['Time'], utc=True)
    
    # Set timestamp as index for both DataFrames
    apy_df = apy_df.set_index('timestamp')
    pt_df = pt_df.set_index('Time')
    
    # Resample APY to 4-hour periods and calculate OHLC for impliedApy
    apy_4h = apy_df['impliedApy'].resample('4H').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last'
    })
    
    # Resample PT volume to 4-hour periods (sum of volumes)
    volume_4h = pt_df['Volume'].resample('4H').sum()
    
    # Join the OHLC data with the volume data
    df_4h = pd.concat([apy_4h, volume_4h], axis=1)
    
    return df_4h


In [7]:
def calculate_ema(data, period = 20):
    return data.ewm(span=period, adjust=False).mean()

def calculate_rsi(data, period=7):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def prepare_dataframe(df):
    """Prepare the DataFrame by setting the correct index and datetime format."""
    if not isinstance(df.index, pd.DatetimeIndex):
        if 'Time' in df.columns:
            df = df.set_index('Time')
        df.index = pd.to_datetime(df.index)
    return df

def create_chart(df, title):
    """Create a candlestick chart with EMA, RSI, and volume."""
    # Calculate EMA 20 and RSI 7
    df['EMA20'] = calculate_ema(df['Close'], 20)
    df['RSI7'] = calculate_rsi(df['Close'], 7)

    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.03, subplot_titles=('', 'RSI 7', 'Volume'), 
                        row_heights=[0.6, 0.2, 0.2])

    # Add OHLC candlesticks
    fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='OHLC'
    ), row=1, col=1)

    # Add EMA 20
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['EMA20'],
        line=dict(color='orange', width=2),
        name='EMA 20'
    ), row=1, col=1)

    # Add RSI 7
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['RSI7'],
        line=dict(color='purple', width=2),
        name='RSI 7'
    ), row=2, col=1)

    # Add RSI overbought/oversold lines
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

    # Add volume bars
    colors = ['red' if row['Open'] > row['Close'] else 'green' for index, row in df.iterrows()]
    fig.add_trace(go.Bar(
        x=df.index, 
        y=df['Volume'],
        marker_color=colors,
        name='Volume'
    ), row=3, col=1)

    # Update layout
    fig.update_layout(
        title=title,
        height=1000,  # Increased height to accommodate RSI
        width=1200,
        showlegend=True,
        paper_bgcolor='white',
        plot_bgcolor='white',
        font=dict(color='black'),
        xaxis=dict(
            title='Date',
            gridcolor='#EEEEEE',
            rangeslider=dict(visible=False)
        ),
        xaxis2=dict(
            title='Date',
            gridcolor='#EEEEEE'
        ),
        xaxis3=dict(
            title='Date',
            gridcolor='#EEEEEE'
        ),
        legend=dict(
            x=0,
            y=1,
            traceorder="normal",
            font=dict(
                family="sans-serif",
                size=12,
                color="black"
            ),
            bgcolor="white",
            bordercolor="black",
            borderwidth=2
        )
    )

    fig.update_yaxes(title_text='Price', row=1, col=1, gridcolor='#EEEEEE')
    fig.update_yaxes(title_text='RSI', row=2, col=1, gridcolor='#EEEEEE')
    fig.update_yaxes(title_text='Volume', row=3, col=1, gridcolor='#EEEEEE')

    return fig

def resample_to_daily(df):
    """Resample the DataFrame to daily timeframe."""
    return df.resample('1D').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })

def resample_to_4h(df):
    """Resample the DataFrame to daily timeframe."""
    return df.resample('4H').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })

def create_and_save_charts(df, base_filename):
    """Create and save both 1-hour and 1-day charts."""
    # Prepare the DataFrame
    df = prepare_dataframe(df)

    # Create 1-hour timeframe chart with EMA
    fig_1h = create_chart(df, f'Trading Chart with EMA 20 (1-Hour Timeframe) - {base_filename}')

    # Resample to 1-day timeframe
    df_1d = resample_to_daily(df)

    # Resample to 1-day timeframe
    df_4h = resample_to_4h(df)

    # Create 1-day timeframe chart with EMA
    fig_1d = create_chart(df_1d, f'Trading Chart with EMA 20 (1-Day Timeframe) - {base_filename}')

    # Create 4-hour timeframe chart with EMA
    fig_4h = create_chart(df_4h, f'Trading Chart with EMA 20 (4-Hour Timeframe) - {base_filename}')

    # Show both charts
    fig_1h.show()
    fig_4h.show()
    fig_1d.show()
    # Save the charts to HTML files
    fig_1h.write_html(f"trading_chart_with_ema20_1h_{base_filename}.html")
    fig_1d.write_html(f"trading_chart_with_ema20_1d_{base_filename}.html")
    fig_4h.write_html(f"trading_chart_with_ema20_4h_{base_filename}.html")
    print(f"Charts saved as 'trading_chart_with_ema20_1h_{base_filename}.html' and 'trading_chart_with_ema20_1d_{base_filename}.html'.")



In [8]:
def calculate_ema(data, period = 20):
    return data.ewm(span=period, adjust=False).mean()

def calculate_rsi(data, period=7):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def detect_divergence(price, rsi, window=5, forward_window=5):
    divergences = []
    for i in range(window, len(price) - forward_window):
        price_window = price[i-window:i+1]
        rsi_window = rsi[i-window:i+1]
        
        price_forward = price[i+2:i+1+forward_window]  # Start 2 periods later
        rsi_forward = rsi[i+2:i+1+forward_window]  # Start 2 periods later
        
        # Check if RSI is within the valid range (5 <= RSI <= 95)
        if 5 <= rsi_window.iloc[-1] <= 95:
            if price_window.iloc[-1] > price_window.iloc[:-1].max():  # New high in price
                if rsi_window.iloc[-1] <= rsi_window.iloc[:-1].max():  # Not a new high in RSI
                    if len(price_forward) >= 2 and price_forward.max() > price_window.iloc[-1]:  # Price continues to make new highs
                        if rsi_forward.max() <= rsi_window.iloc[-1]:  # RSI doesn't make new highs
                            if rsi_window.iloc[-1] > 65:  # RSI is above 65
                                divergences.append((i))
            
            elif price_window.iloc[-1] < price_window.iloc[:-1].min():  # New low in price
                if rsi_window.iloc[-1] >= rsi_window.iloc[:-1].min():  # Not a new low in RSI
                    if len(price_forward) >= 2 and price_forward.min() < price_window.iloc[-1]:  # Price continues to make new lows
                        if rsi_forward.min() >= rsi_window.iloc[-1]:  # RSI doesn't make new lows
                            if rsi_window.iloc[-1] < 35:  # RSI is below 35
                                divergences.append((i))
    
    return divergences
def create_chart(df, title):
    # Calculate EMA 20 and RSI 14
    df['EMA20'] = calculate_ema(df['Close'], 20)
    df['RSI7'] = calculate_rsi(df['Close'], 7)
    
    # Detect divergences
    divergences = detect_divergence(df['Close'], df['RSI7'])

    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.03, subplot_titles=('', 'RSI 7', 'Volume'), 
                        row_heights=[0.6, 0.2, 0.2])

    # Add OHLC candlesticks
    fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='OHLC'
    ), row=1, col=1)

    # Add EMA 20
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['EMA20'],
        line=dict(color='orange', width=2),
        name='EMA 20'
    ), row=1, col=1)

    # Add RSI 14
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['RSI7'],
        line=dict(color='purple', width=2),
        name='RSI 14'
    ), row=2, col=1)

    # Add RSI overbought/oversold lines
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

    # Add volume bars
    colors = ['red' if row['Open'] > row['Close'] else 'green' for index, row in df.iterrows()]
    fig.add_trace(go.Bar(
        x=df.index, 
        y=df['Volume'],
        marker_color=colors,
        name='Volume'
    ), row=3, col=1)

    # Add divergence markers
    for div in divergences:
        fig.add_shape(
            type="line",
            x0=df.index[div-10], y0=df['Close'].iloc[div-10],
            x1=df.index[div], y1=df['Close'].iloc[div],
            line=dict(color="red", width=2, dash="dot"),
            row=1, col=1
        )
        fig.add_annotation(
            x=df.index[div], y=df['Close'].iloc[div],
            text="Divergence",
            showarrow=True,
            arrowhead=2,
            row=1, col=1
        )

    # Update layout
    fig.update_layout(
        title=title,
        height=1000,
        width=1200,
        showlegend=True,
        paper_bgcolor='white',
        plot_bgcolor='white',
        font=dict(color='black'),
        xaxis=dict(
            title='Date',
            gridcolor='#EEEEEE',
            rangeslider=dict(visible=False)
        ),
        xaxis2=dict(
            title='Date',
            gridcolor='#EEEEEE'
        ),
        xaxis3=dict(
            title='Date',
            gridcolor='#EEEEEE'
        ),
        legend=dict(
            x=0,
            y=1,
            traceorder="normal",
            font=dict(
                family="sans-serif",
                size=12,
                color="black"
            ),
            bgcolor="white",
            bordercolor="black",
            borderwidth=2
        )
    )

    fig.update_yaxes(title_text='Price', row=1, col=1, gridcolor='#EEEEEE')
    fig.update_yaxes(title_text='RSI', row=2, col=1, gridcolor='#EEEEEE')
    fig.update_yaxes(title_text='Volume', row=3, col=1, gridcolor='#EEEEEE')

    return fig

def resample_to_daily(df):
    """Resample the DataFrame to daily timeframe."""
    return df.resample('1D').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })

def resample_to_4h(df):
    """Resample the DataFrame to daily timeframe."""
    return df.resample('4H').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })

def create_and_save_charts(df, base_filename):
    """Create and save both 1-hour and 1-day charts."""
    # Prepare the DataFrame
    df = prepare_dataframe(df)

    # Create 1-hour timeframe chart with EMA
    fig_1h = create_chart(df, f'Trading Chart with EMA 20 (1-Hour Timeframe) - {base_filename}')

    # Resample to 1-day timeframe
    df_1d = resample_to_daily(df)

    # Resample to 1-day timeframe
    df_4h = resample_to_4h(df)

    # Create 1-day timeframe chart with EMA
    fig_1d = create_chart(df_1d, f'Trading Chart with EMA 20 (1-Day Timeframe) - {base_filename}')

    # Create 4-hour timeframe chart with EMA
    fig_4h = create_chart(df_4h, f'Trading Chart with EMA 20 (4-Hour Timeframe) - {base_filename}')

    # Show both charts
    fig_1h.show()
    fig_4h.show()
    fig_1d.show()
    # Save the charts to HTML files
    fig_1h.write_html(f"trading_chart_with_ema20_1h_{base_filename}.html")
    fig_1d.write_html(f"trading_chart_with_ema20_1d_{base_filename}.html")
    fig_4h.write_html(f"trading_chart_with_ema20_4h_{base_filename}.html")
    print(f"Charts saved as 'trading_chart_with_ema20_1h_{base_filename}.html' and 'trading_chart_with_ema20_1d_{base_filename}.html'.")

create_and_save_charts(df, "PT")

C:\Users\DC\AppData\Local\Temp\ipykernel_29220\4078586692.py:160: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



Charts saved as 'trading_chart_with_ema20_1h_PT.html' and 'trading_chart_with_ema20_1d_PT.html'.


In [9]:
# Convert APY to 4-hour OHLC data and join with PT volume
APY_4h = convert_to_ohlc_4h(APY, df)
create_and_save_charts(APY_4h, "APY")

C:\Users\DC\AppData\Local\Temp\ipykernel_29220\2235466560.py:35: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

C:\Users\DC\AppData\Local\Temp\ipykernel_29220\2235466560.py:43: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

C:\Users\DC\AppData\Local\Temp\ipykernel_29220\4078586692.py:160: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



Charts saved as 'trading_chart_with_ema20_1h_APY.html' and 'trading_chart_with_ema20_1d_APY.html'.
